---
---

> <center><img src="https://rapids.ai/assets/images/og_image.png"></center>

---
---

In [ ]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Training Datasets
train_feature= pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_targets_scored= pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
train_targets_nonscored= pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')

In [ ]:
# Test & Sample Submission
test_features= pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
sample_sub=pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

---
---

> <center><font color="red"><h1 style="text-transform: uppercase; text-shadow: 1px 1px;"><b>Little Data Exploration <font></font> </b></h1></center>


---


In [ ]:
train_feature.head()

In [ ]:
test_features.head()

In [ ]:
train_targets_nonscored.head()

In [ ]:
train_targets_scored.head()

In [ ]:
sample_sub.head()

In [ ]:
print(train_feature.shape)
print(train_targets_scored.shape)
print(train_targets_nonscored.shape)
print(test_features.shape)

---
---

> <center><font color="red"><h1 style="text-transform: uppercase; text-shadow: 1px 1px;"><b>Install RAPIDS 0.15.0 <font></font> </b></h1></center>


---


In [ ]:
import sys
!cp ../input/rapids/rapids.0.15.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.7/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.7"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path 
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [ ]:
import cupy
import cudf
from cuml.neighbors import KNeighborsClassifier, NearestNeighbors
import cuml; cuml.__version__

---
---

> <center><font color="red"><h1 style="text-transform: uppercase; text-shadow: 1px 1px;"><b>Data Pre-Processing<font></font> </b></h1></center>


---


In [ ]:
train_features = pd.read_csv('../input/lish-moa/train_features.csv')
train_targets = pd.read_csv('../input/lish-moa/train_targets_scored.csv')

In [ ]:
# taken from https://www.kaggle.com/stanleyjzheng/baseline-nn-with-k-folds
COLS = ['cp_type','cp_dose']
FE = []
for col in COLS:
    for mod in train_features[col].unique():
        FE.append(mod)
        train_features[mod] = (train_features[col] == mod).astype(int)
del train_features['sig_id']
del train_features['cp_type']
del train_features['cp_dose']
FE+=list(train_features.columns) 
del train_targets['sig_id']

print(np.array(train_features.to_numpy(), dtype=np.float).shape)
print(np.array(train_targets.to_numpy(), dtype=np.float).shape)

In [ ]:
# taken from https://www.kaggle.com/stanleyjzheng/baseline-nn-with-k-folds

test_features = pd.read_csv('../input/lish-moa/test_features.csv')


for col in COLS:
    for mod in test_features[col].unique():
        test_features[mod] = (test_features[col] == mod).astype(int)

sig_id = pd.DataFrame()
sig_id = test_features.pop('sig_id')
del test_features['cp_type']
del test_features['cp_dose']

columns = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
del columns['sig_id']

---
---

> <center><font color="red"><h1 style="text-transform: uppercase; text-shadow: 1px 1px;"><b>Building Model<font></font> </b></h1></center>


---


In [ ]:
from sklearn.model_selection import train_test_split, KFold

# Train test split # CREATE 20% VALIDATION SET
X_train, X_test, y_train, y_test = train_test_split(train_features, train_targets ,test_size=0.2, random_state=42)

In [ ]:
from cuml.neighbors import KNeighborsClassifier, NearestNeighbors

# GRID SEARCH FOR OPTIMAL K
accs = []
for k in range(5,50):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    y_hat_p = knn.predict(X_test)
    print(k,', ',end='')

---
---

> <center><img src="https://www.websystemer.no/wp-content/uploads/2020/08/beginners-guide-to-knn-with-cuml.gif"></center>

---
---

In [ ]:
y_hat_test = knn.predict(test_features)
sub = pd.DataFrame(data=y_hat_test, columns=columns.columns)

---
---

> <center><font color="red"><h1 style="text-transform: uppercase; text-shadow: 1px 1px;"><b>Submission<font></font> </b></h1></center>


---


In [ ]:
sample = pd.read_csv('../input/lish-moa/sample_submission.csv')
sub.insert(0, column = 'sig_id', value=sample['sig_id'])

In [ ]:
sub.to_csv('submission.csv', index=False)

In [ ]:
sub.head()

For some bug I'm not able to submit!

Issue: https://www.kaggle.com/c/lish-moa/discussion/180114